In [1]:
import dspy
import spacy
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt 
from datasets import load_dataset
import random
from sentence_transformers import SentenceTransformer, util




/Users/sntn/opt/anaconda3/envs/left/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
dataset = load_dataset("code_search_net", "java")

system_prompt = "create documentation from the code in the question using the example answers that are given"

# Set up the LM
#turbo = dspy.OllamaLocal(model='', max_tokens=250)
#turbo = dspy.OllamaLocal(model='llamca3')
#turbo = dspy.OllamaLocal(model='mistral')
turbo = dspy.OllamaLocal(model='llama3')
#ßturbo = dspy.OllamaLocal(model='codegemma:code',system_prompt=system_prompt)
#turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=250)

dspy.settings.configure(lm=turbo)


/Users/sntn/opt/anaconda3/envs/left/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
import TreeSearch
kcases = 10
training_data=[]
for x in range(kcases):
    k=random.randrange(454450)
    case=dspy.Example(question=dataset['train'][k]['func_code_string'], answer=dataset['train'][k]['func_documentation_string']).with_inputs('question')
    training_data.append(case)
treesearcher=TreeSearch.TreeSearch(model=turbo, pruning_threshold=0.01, pruning_delay=10,max_rounds=30,
                                   max_labeled_demos=8, sparsity_demos=0.5, training_set = training_data, breadth = 10 )
treesearcher.test_save()

In [3]:
#make training set 
kcases = 10
training_data=[]
for x in range(kcases):
    k=random.randrange(454450)
    case=dspy.Example(question=dataset['train'][k]['func_code_string'], answer=dataset['train'][k]['func_documentation_string']).with_inputs('question')
    training_data.append(case)
    
#make test set 


kcases = 10
test_data=[]
for x in range(kcases):
    k=random.randrange(454450)
    case=dspy.Example(question=dataset['train'][k]['func_code_string'], answer=dataset['train'][k]['func_documentation_string']).with_inputs('question')
    test_data.append(case)

In [4]:
def sentenceSimilarity(example, pred,  trace=None):
    example_arr = [example]
    pred_arr = [pred]
    example_embedding = model.encode(example_arr, convert_to_tensor=True)
    prediction_embedding = model.encode(pred_arr, convert_to_tensor=True)
    return(util.pytorch_cos_sim(example_embedding, prediction_embedding).item())

#model = SentenceTransformer("all-MiniLM-L6-v2")
#model = SentenceTransformer('stsb-roberta-large')
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

similarity = []

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/Users/sntn/opt/anaconda3/envs/left/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import TreeSearch
treesearcher=TreeSearch.TreeSearch(model=turbo, pruning_threshold=0.01, pruning_delay=10,max_rounds=50,
                                   max_labeled_demos=8, sparsity_demos=0.5, training_set = training_data, breadth = 10 )
highest_so_far, highest_fitness=treesearcher.compile(sentenceSimilarity,training_data)

Done in 1:01 at 20:43:05/10 0:07min/perc 0.14it/s | 1:01:06 => 1:00
Done in 0:57 at 20:44:02/10 0:05min/perc 0.18it/s | 0:57:05 => 0:57
Done in 0:57 at 20:45:00/10 0:06min/perc 0.18it/s | 0:57:06 => 0:57
Done in 0:58 at 20:46:45/10 0:06min/perc 0.18it/s | 0:58:06 => 0:58
Done in 0:58 at 20:47:42/10 0:06min/perc 0.18it/s | 0:58:06 => 0:58
Done in 6:53 at 20:47:56/10 0:14min/perc 0.07it/s | 6:53:28 => 0:4227
round 0fitness = 0.338028146462007
Done in 0:54 at 20:48:50/10 0:05min/perc 0.19it/s | 0:54:05 => 0:5444:04
Done in 0:17 at 20:49:07/10 0:02min/perc 0.66it/s | 0:17:02 => 0:17
Done in 0:57 at 20:50:35/10 0:05min/perc 0.19it/s | 0:57:05 => 0:57:27
Done in 0:50 at 20:51:25/10 0:05min/perc 0.21it/s | 0:50:05 => 0:50
Done in 0:59 at 20:52:24/10 0:06min/perc 0.18it/s | 0:59:06 => 0:59
Done in 0:39 at 20:53:03/10 0:04min/perc 0.28it/s | 0:39:04 => 0:39
Done in 0:58 at 20:54:17/10 0:06min/perc 0.18it/s | 0:58:06 => 0:58
Done in 6:21 at 20:54:17/10 0:58min/perc 0.02it/s | 6:21
round 1fitness

In [ ]:
%pip install tqdm


In [ ]:
prompt = "hello"
#turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=250)
turbo = dspy.OllamaLocal(model='llama3')
response = turbo.request(prompt)
print (response["choices"][0]['message']['content'])

In [ ]:
def create_heatmap(similarity, labels, cmap = "YlGnBu"):
  df = pd.DataFrame(similarity)
  df.columns = labels
  df.index = labels
  fig, ax = plt.subplots(figsize=(5,5))
  sns.heatmap(df, cmap=cmap)



def heatmap(training_data):
    nlp = spacy.load('en_core_web_md')
    docs = [nlp(n.answer) for n in training_data]
    labels = [training_data.answer[:20] for training_data in training_data]
    similarity = []
    for i in range(len(docs)):
        row = []
        for j in range(len(docs)):
          row.append(docs[i].similarity(docs[j]))
        similarity.append(row)
    create_heatmap(similarity,labels)

In [ ]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)
    
class Pred(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)    
    

In [ ]:
#use a spacy embedding model to compare the example's answer to the classifiers prediction
def validate_answer(example, pred, trace=None):
    nlp = spacy.load('en_core_web_md')
    print (pred)
    answer_vec=nlp(example.answer)
    pred_vec=nlp(pred.answer)
    return (answer_vec.similarity(pred_vec))

embeddings=[]



def sentenceSimilarity(example, pred,  trace=None):
    example_arr = [example.answer]
    pred_arr = [pred.answer]
    example_embedding = model.encode(example_arr, convert_to_tensor=True)
    prediction_embedding = model.encode(pred_arr, convert_to_tensor=True)
    return(util.pytorch_cos_sim(example_embedding, prediction_embedding).item())

#model = SentenceTransformer("all-MiniLM-L6-v2")
#model = SentenceTransformer('stsb-roberta-large')
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')


exp = dspy.Example(question="some question", answer="This is an indentical sentence")
prd = dspy.Example(question="another question", answer="This is an indentical sentence")

print(sentenceSimilarity(exp,prd))
prd = dspy.Example(question="another question", answer="This is a different sentence")
print(sentenceSimilarity(exp,prd))
prd = dspy.Example(question="another question", answer="zookeepers and elephants dance the night away")
print(sentenceSimilarity(exp,prd))
prd = dspy.Example(question="another question", answer="This is an indenticl sentence")
print(sentenceSimilarity(exp,prd))
prd = dspy.Example(question="another question", answer="This is an identical phrase")
print(sentenceSimilarity(exp,prd))


similarity = []
#for i in range(len(sentences)):
 #   row = []
   # for j in range(len(sentences)):
   #   row.append(util.pytorch_cos_sim(embeddings[i], embeddings[j]).item())
#similarity.append(row)
    
#create_heatmap(similarity)

In [ ]:
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch,BootstrapFewShotWithOptuna,MIPRO, LabeledFewShot
from dspy.teleprompt.vanilla import LabeledFewShot
#model = SentenceTransformer('stsb-roberta-large')
#model = SentenceTransformer("all-MiniLM-L6-v2")
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

#trainModel()

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 4-shot examples of our CoT program.
#config = dict(max_bootstrapped_demos=8, max_labeled_demos=8)
config = dict()

#teleprompter = BootstrapFewShotWithRandomSearch(metric=sentenceSimilarity,num_candidate_programs=20, **config)

#teleprompter = BootstrapFewShotWithOptuna(metric=sentenceSimilarity,num_candidate_programs=10, **config)
#teleprompter = MIPRO(metric=sentenceSimilarity, **config)

teleprompter = LabeledFewShot(metric=sentenceSimilarity, **config)
optimized_pred = teleprompter.compile(Pred(), trainset=training_data, max_demos=8)
#teleprompter = BootstrapFewShot(metric=validate_answer, **config)
#optimized_cot = teleprompter.compile(CoT(), trainset=training_data)
#optimized_pred = teleprompter.compile(Pred(), trainset=training_data, max_demos=8)
#optimized_pred = teleprompter.compile(Pred(), trainset=training_data, num_trials=50, max_bootstrapped_demos=1, 
            #                            max_labeled_demos=8,eval_kwargs=config,requires_permission_to_run=False)

In [ ]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=training_data, metric=sentenceSimilarity, num_threads=4, display_progress=True, display_table=0)
test = Evaluate(devset=test_data, metric=sentenceSimilarity, num_threads=4, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
#evaluate(optimized_pred)


In [ ]:
print("unoptimised training performance")
evaluate (Pred())


In [ ]:
print("optimised training performance")
evaluate(optimized_pred)

In [ ]:
print("unoptimised test performance")
test (Pred())

In [ ]:
print("optimised test performance")
test (optimized_pred)

In [ ]:
#optimized_cot(question='protected String getTimestamp() {Date date = new Date(System.currentTimeMillis()); DateFormat formatter = DateFormat.getDateTimeInstance(DateFormat.SHORT, DateFormat.SHORT);  String finishTime = formatter.format(date); return finishTime;}')

optimized_pred(question='public void actionPerformed(ActionEvent evt) {Object source = evt.getSource(); // --- look and feelif ( source == nativeLF   ) {getNativeUI();return;}else if ( source == metalLF   ) {try {System.out.println("Metal look & feel chosen");UIManager.setLookAndFeel(UIManager.getCrossPlatformLookAndFeelClassName());SwingUtilities.updateComponentTreeUI(this); }catch (Exception exc) {System.out.println("Error getting UI");}return;}// others if (context != null) {if ( source == exit ) exitBtnFn();else if ( source == tile   ) bottomPanel.tile();else if ( source == cascade   ) bottomPanel.cascade(); else if ( source == about   ) { }else if (source == mailInBtn) {new MailInTableUI(bottomPanel,mailInBuffer);} else if (source == mailOutBtn) { new MailOutTableUI(bottomPanel,mailOutBuffer);}else if (source == msgHandlerBtn) { new MsgHandlerTableUI(bottomPanel,msgHandlerBuffer); } }  else { JOptionPane.showMessageDialog(this,"No associated agent","Error Message", JOptionPane.OK_OPTION); }')



In [ ]:
optimized_pred.__dict__

In [ ]:
optimized_pred.prog.signature.

In [ ]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=training_data, metric=sentenceSimilarity, num_threads=4, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(Pred())

In [ ]:
turbo.inspect_history(n=1)
